<a href="https://colab.research.google.com/github/dung18520632/CS114.K21/blob/master/Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount( '/content/gdrive' )

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import cv2 
import time
import pickle
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def Danh_Lable(Y, path, sl = -1):
    if sl == -1:
      sl = len(os.listdir(path))

    for pic in os.listdir(path):
      if pic[:3] == 'pos':
        Y.append(1)
      else:
        if  pic[:3] == 'neg':
          Y.append(0)
        else:
          Y.append(-1)
      sl = sl - 1
      if sl <= 0:
        break
    return Y

def load_img(link, i,sig=1):
    if sig == 1:
      print("Loading image {} : {}".format(i + 1, link))
    t0 = time.time()
    img = cv2.imread(link)
    t1 = time.time()
    if sig == 1:
      print("Time : {0:.3f}".format(t1-t0))
    return img

def detect_faces(img,sig=1):
    if sig == 1:
      print("START DETECT FACE",end="")
      print("-"*63)

    t0 = time.time()
    cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = cascade.detectMultiScale(img)
    if sig == 1:
      print("- Has found {} faces".format(len(faces)))

    if len(faces) != 0:
        (x, y, w, h) = faces[0]
        r = max(w, h) / 2
        centerx = x + w / 2
        centery = y + h / 2
        nx = int(centerx - r)
        ny = int(centery - r)
        nr = int(r * 2)
        faceimg = img[ny:ny+nr, nx:nx+nr]
        t1 = time.time()
        if sig == 1:
          print("- Time : {0:.3f}".format(t1-t0))
        return faceimg
    else:
        if sig == 1:
          print("NO FACE DETECED")
          print("LOAD NEXT IMAGE")
          t1 = time.time()
          print("- Time : {0:.3f}".format(t1-t0))
        return None

def resize(img, w=50, h=50,sig=1):
    if sig == 1:
      print("START RESIZE ",end="")
      print("-"*67)
    img_resized = cv2.resize(src=img, dsize=(w, h))
    return img_resized

def convert(img,sig=1):
    if sig == 1:
      print("START CONVERT COLOR ",end="")
      print("-"*60)
    image_t = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return image_t

def crop_face(link, sl=-1, sig=1):
    if sig == 1:
      print("START CROPPING FACE")
      print("="*80)

    list_image = []
    list_os = os.listdir(link)
    if sl == -1:
      sl = len(list_os)
    
    for i, name in enumerate(list_os[:sl]):
      if sig == 1:
        print("_"*80)
      # Load anh
      path = link + "/" + name
      img = load_img(path, i,sig)
      # Detect face
      face_cvted = convert(img,sig)
      face = detect_faces(face_cvted,sig)
      if face is None:
        list_image.append(None)
        continue;
      # Resize
      face_resize = resize(face,sig=sig)
      # Convert
      #os.chdir(r'C:\Users\Laptop\Desktop\DoAnCuoiky\train convert') 
      #cv2.imwrite(name, face_resize)
      list_image.append(face_resize)
    return list_image

def create_feature(data):
  zero = np.array([0]*2500)

  feature = []
  for pic in data:
    if pic is None:
      feature.append(zero)
    else:
      feature.append(pic.flatten() / 255)
  return feature


def split_data(X, Y, alpha = 0.5):
  edge = int(len(X) * alpha)
  features = []
  target = []
  index = np.random.permutation(len(X))
  for i in index:
      features.append(X[i])
      target.append(Y[i])
  X_train = features[:edge]
  X_test = features[edge:]

  Y_train = target[:edge]
  Y_test = target[edge:]
  return (X_train, Y_train, X_test, Y_test)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims = True)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def saveModel(model,link=None):
    if link == None:
        with open("pickle.txt", "wb") as f:
            pickle.dump(model, f)
    else:
         with open(link + "\pickle.txt", "wb") as f:
            pickle.dump(model, f)
            
def openModel(link=None):
    if link == None:
        with open("pickle.txt", "rb") as f:
            p = pickle.load(f)
    else:
        with open(link + "\pickle.txt", "rb") as f:
            p = pickle.load(f)
    return p

In [0]:
folder = '/content/gdrive/My Drive/Colab Notebooks/train'
M = []
M =  Danh_Lable(M, folder)
faces = crop_face(folder)
feature = create_feature(faces)
X_train, Y_train, X_test, Y_test = split_data(feature, M, 0.8)

In [0]:
clfcv = LogisticRegressionCV(cv=5,max_iter=100, tol=1e-4,solver='liblinear')
clfcv.fit(X_train, Y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='liblinear', tol=0.0001, verbose=0)

In [0]:
Y_predict = clfcv.predict(X_test)

cf_mx = confusion_matrix(Y_test, Y_predict)
print("Accuracy : ",accuracy_score(Y_test, Y_predict))

Accuracy :  0.6818181818181818


In [0]:
saveModel(clfcv)

In [0]:
folder_pred = '/content/gdrive/My Drive/Colab Notebooks/predict'
clf = openModel()
real = []

real = Danh_Lable(real,folder_pred)
img_pred = crop_face(folder_pred,sig=0)
img_pred = create_feature(img_pred)
ketqua = clf.predict(img_pred)
print("Accuracy : ",accuracy_score(real, ketqua))

Accuracy :  0.5714285714285714
